In [2]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [3]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv(' clean_fifa_worldcup_fixture.csv')

Calcularemos la eficiencia de un equipo dado los goles que mete contra los goles que recibe


In [4]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [5]:
#aqui definimos tanto los foles anotados como los recibidos tanto para el equipo local como para el visitante.
df_home=df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away=df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [6]:
#concatenamos df_home y df_away, hacemos grupos y calculamos el promedio de goles de los equipos.
df_team_strength =pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

A continuacion usaremos la funcion de predicción basada en poisson

In [7]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)


In [8]:
#probamos la funcion con equipos y calculamos las probabilidades
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))


(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)


Prediccion de la copa del mundo.

Primero empezzmos con la fase de grupos ahora dividimos de los equipos sus enfrentamientos 
    

In [30]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()


In [33]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)


#Aqui basicamente para todos los grupos obtenemos los equipos y de los 48 partidos de fase de grupos
#dividimos o filtramos por grupo con sus respectivos partidos (6 por grupo)

In [39]:
dict_table['Group A']

#Aqui simulamos los puntos que pueden obtener los equipos dado los resultados simulados


,Team,Pts
0,Netherlands,4.0
1,Senegal,2.0
2,Ecuador,2.0
3,Qatar (H),0.0


Ahora dividimos los octavos de final, y ahora tendremos que elegir a los equipos que pasarían de ronda dado sus resultados por grupo

In [43]:
df_fixture_knockout

for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

#estos serán los posibles enfrentamientos de octavos de final (primer y segundo lugar de cada grupo)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [44]:
#Esta función nos ayuda a opbetenr al posible ganador de los enfrentaminetos de octavos

def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated


In [46]:
get_winner(df_fixture_knockout)
#aqui obtenemos al posible ganador

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


CUARTOS DE FINAL

Aqui ahora de los posibles ganadores, obtendremos a los 8 equipos que nos regresa y los organizamos para saber los enfrentamientos de cuartos de final.

In [47]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2


In [48]:
update_table(df_fixture_knockout, df_fixture_quarter)
#posibles enfrentamientos de cuartos.

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [49]:
#Obtenemos ahora al posible ganador de cada ronda de los cuartos de final.
get_winner(df_fixture_quarter)


,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


SEMIFINAL

Ahora predecimos de la misma manera pero ahora para los 4 equipos restantes.

In [50]:
update_table(df_fixture_quarter, df_fixture_semi)


,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [51]:
get_winner(df_fixture_semi)


,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


Final 
Hacemos el mismo procedimiento ahora para la final

In [52]:
update_table(df_fixture_semi, df_fixture_final)



,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [53]:
get_winner(df_fixture_final)


,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
